In [1]:
with open('input.txt', "r") as file:
    data = file.read().split('\n\n')

print(data[0])

departure location: 27-374 or 395-974
departure station: 40-287 or 295-953
departure platform: 27-554 or 570-961
departure track: 40-604 or 618-958
departure date: 43-842 or 850-972
departure time: 30-302 or 315-952
arrival location: 32-478 or 496-950
arrival station: 48-733 or 755-969
arrival platform: 37-260 or 276-954
arrival track: 40-512 or 519-964
class: 34-277 or 284-966
duration: 25-648 or 672-961
price: 28-684 or 705-956
route: 30-157 or 176-950
row: 47-881 or 903-970
seat: 38-705 or 727-959
train: 40-195 or 217-961
type: 28-858 or 879-958
wagon: 31-543 or 554-967
zone: 49-790 or 816-953


In [2]:
test_0 = '''class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12'''.split('\n\n')

In [3]:
test_1 = '''class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9'''.split('\n\n')

# Part 1

In [4]:
def parse_data(ticket_rules, your_ticket, nearby_tickets):
    '''
    Variables:
        rules -> {rule_name: [a1, b1, a2, b2]}
            a1 <= x <= b1 or a1 <= x <= b1 ~ rule is satisfied
        your_ticket -> [int, ...]
        nearby_tickets -> [[int, ...], ...]

    Returns:
        (rules, your_ticket, nearby_tickets)
    '''
    ticket_rules = ticket_rules.strip().split('\n')
    rules = {}
    for rule in ticket_rules:
        rule_title, rule_values = rule.split(': ')
        range_1, range_2 = rule_values.split(' or ')
        r1_a, r1_b, r2_a, r2_b = list(map(int, (*range_1.split('-'), *range_2.split('-'))))
        rules[rule_title] = (r1_a, r1_b, r2_a, r2_b)
    your_ticket = list(map(int, your_ticket.strip().split('\n')[1].split(',')))
    nearby_tickets = list(map(
        lambda x: list(map(lambda y: int(y), x.split(','))),
        nearby_tickets.strip().split('\n')[1:]))

    return rules, your_ticket, nearby_tickets

def compute_error(nearby_tickets, rules):
    not_valid = []
    for ticket_vals in nearby_tickets:
        for val in ticket_vals:
            valid = False
            for a1, b1, a2, b2 in rules.values():
                if a1 <= val <= b1 or a2 <= val <= b2:
                    valid = True
                    break
            if not valid:
                not_valid.append(val)
    return(sum(not_valid))

def solve_p1(data):
    rules, your_ticket, nearby_tickets = parse_data(*data)
    res = compute_error(nearby_tickets, rules)
    return res


assert(solve_p1(test_0)) == 71

solve_p1(data)

26941

# Part 2

In [5]:
from functools import reduce

def get_valid_tickets(tickets, rules):
    '''
    Form list from tickets, where each element 
    satisfies at least one rule
    
    Variables:
        valid_tickets -> [[int, ...], ...]

    Returns:
        valid_tickets
    '''
    ids_not_valid = set()
    for t_id, ticket_vals in enumerate(tickets):
        for val in ticket_vals:
            valid = False
            for a1, b1, a2, b2 in rules.values():
                if a1 <= val <= b1 or a2 <= val <= b2:
                    valid = True
                    break
            if not valid:
                ids_not_valid.add(t_id)

    valid_tickets = [ticket for t_id, ticket in enumerate(tickets) if t_id not in ids_not_valid]
    return valid_tickets

def find_rule2index_mapping(tickets, rules):
    '''
    Form matrix M, such that
    M[row_i][col_j] == 1 <=> rule i stands for col_j
    Add rule names in rows
    Find rule2index mapping
    Note, that 
    exactly one solution <=> there are no 
    
    Variables:
        rule2index_mapping = {
            rule_name: index, ...
        }

    Returns:
        rule2index_mapping
    '''
    M = [[1 for j in range(len(rules))]
         for i in range(len(rules))]

    for i, ticket_vals in enumerate(tickets):
        for q, val in enumerate(ticket_vals):
            for p, rule_name in enumerate(rules):
                a1, b1, a2, b2 = rules[rule_name]
                if not (a1 <= val <= b1) and not (a2 <= val <= b2):
                    M[p][q] = 0

    rule2index_map = {}
    named_M = [[line, rule_name] for rule_name, line in zip(rules, M)]
    srtd_nmd_M = sorted(named_M, key = lambda x: sum(x[0]))
    for i, line in enumerate(srtd_nmd_M):
        # as M consists only of 1s and 0s
        # only one 1 is guaranteed for solvable 
        if sum(line[0]) == 1:
            col_i = line[0].index(1)
            for row_i in range(i+1, len(srtd_nmd_M)):
                srtd_nmd_M[row_i][0][col_i] = 0
            rule2index_map[line[1]] = line[0].index(1)
        else:
            rule2index_map = None
            break

    return rule2index_map

def solve_p2(data):
    rules, your_ticket, nearby_tickets = parse_data(*data)
    valid_tickets = get_valid_tickets(nearby_tickets, rules)

    rule2index_map = find_rule2index_mapping(valid_tickets, rules)
    indxs = [rule2index_map[rule_name] for rule_name in rule2index_map if 'departure' in rule_name]

    return reduce(lambda a, b: a*b, [your_ticket[i] for i in indxs])

def test_mapping(data, mapping):
    rules, your_ticket, nearby_tickets = parse_data(*data)
    valid_tickets = get_valid_tickets(nearby_tickets, rules)
    rule2index_map = find_rule2index_mapping(valid_tickets, rules)
    res = True
    for rule in rule2index_map:
        if rule2index_map[rule] != mapping[rule]:
            res = False
            break
    return res


assert(test_mapping(test_1, {'class': 1, 'row': 0, 'seat': 2}))

solve_p2(data)

634796407951